In [13]:
# !pip install yt_dlp
# !pip install jiwer
# !pip install pyngrok
# !pip install flask-cors
# !pip install nltk


# First installation of modules and importing, the required ones

In [1]:
# import yt_dlp
# #from pydub import AudioSegment
# import time
# #transcription
# import jiwer
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
# import torch
# from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# import yt_dlp
# from pydub import AudioSegment
# import threading
# import requests
# import gdown
# import os

import importlib
import subprocess
import sys

# Function to check and install modules if not already installed
def install_and_import(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    finally:
        globals()[package] = importlib.import_module(package)

# List of required packages
required_packages = [
    "yt_dlp",
    "jiwer",
    "pyngrok",
    "flask_cors",
    "nltk",
    "pydub",
    "torch",
    "transformers",
    "requests",
    "gdown"
]

# Install all required packages
for package in required_packages:
    install_and_import(package)

# Additional setup for nltk
import nltk
nltk.download('punkt')  # Download necessary NLTK data if needed

# Now import all modules
import yt_dlp
import time
import jiwer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from pydub import AudioSegment
import threading
import requests
import gdown
import os

print("All required modules are installed and imported successfully!")


Installing yt_dlp...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 21.8 MB/s eta 0:00:0000:0100:01



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


Installing jiwer...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.8 MB/s eta 0:00:0000:0100:01



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


Installing pyngrok...


...

Installing pydub...



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Installing gdown...



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
E0000 00:00:1742047425.355607      10 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:230


All required modules are installed and imported successfully!


# Validating and setting the environment to cuda GPA usage 

In [2]:

device_use = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype_use = torch.float16 if torch.cuda.is_available() else torch.float32
model_id_use = "openai/whisper-large-v3"
trans_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id_use, torch_dtype=torch_dtype_use, low_cpu_mem_usage=True, use_safetensors=True
)
trans_model.to(device_use)
processor = AutoProcessor.from_pretrained(model_id_use)
pipe = pipeline(
    "automatic-speech-recognition",
    model=trans_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype_use,
    device=device_use,
)




Device set to use cpu
/usr/local/lib/python3.10/site-packages/transformers/pipelines/automatic_speech_recognition.py:312: FutureWarning: `max_new_tokens` is deprecated and will be removed in version 4.49 of Transformers. To remove this warning, pass `max_new_tokens` as a key inside `generate_kwargs` instead.
  warnings.warn(


# All functions for downloading and transcription

In [8]:
def get_youtube_video_details(url):
    # get video detail
    ydl_opts = {
        'quiet': True,  # Suppress unnecessary output
        'noplaylist': True,  # Ensure only a single video is processed
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)  # Extract video info without downloading

    # Extract relevant details
    video_details = {
        "title": info.get("title"),
    #    "uploader": info.get("uploader"),
        "duration": int(info.get("duration", 0)),
    #    "view_count": info.get("view_count"),
    #    "like_count": info.get("like_count"),
    #    "dislike_count": info.get("dislike_count"),  # Sometimes unavailable
    #    "upload_date": info.get("upload_date"),  # YYYYMMDD format
        "thumbnail": info.get("thumbnail"),
    #   "video_url": info.get("webpage_url"),
    #    "description": info.get("description"),
    }
    return video_details

def download_youtube_audio(url, output_file):
    """download video and convert into audio"""
     
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': '/kaggle/working/temp_audio.wav',  # Temporary file path
        'postprocaessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'ffmpeg_location': '/usr/bin/ffmpeg',  # Ensure ffmpeg location is set if necessary
    }

    # Download the audio
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    # Convert to 16kHz, mono WAV
    audio = AudioSegment.from_file("/kaggle/working/temp_audio.wav")
    audio = audio.set_frame_rate(16000).set_channels(1)  # Set to mono if needed
    audio.export(output_file, format="wav")
    print(f"Downloaded and converted to {output_file}")

def transcription_api(id,video_url,api_url ):
    """Main func to call get_video_detail, download video funtcions and transcribe audio"""
     
    video_info = get_youtube_video_details(video_url)
    print(f"video info :{video_info}")
    data_heading = "VideoDetail"
    video_detail_api = api_url + "/api/v1/videos/addVideoDetails"
    return_api(id,data_heading,video_info,video_detail_api)    
    output_file = "/kaggle/working/output_audio.wav"
    start=time.time()
    download_youtube_audio(video_url, output_file)
    download_end=time.time()
    print(f"Download time: {download_end-start} sec")
    transcribe(id,output_file,api_url)
    
def transcribe(id,output_file,ngrok_url):
    print(output_file)
    transcription_api=ngrok_url+"/api/v1/videos/addTranscript"
    generate_kwargs = {
    "return_timestamps": True,
    }
    transcription_start=time.time()
    transcription_s=pipe(output_file, generate_kwargs=generate_kwargs)
    print(transcription_s["chunks"])
    data_heading="original"
    return_api(id,data_heading,transcription_s["chunks"],transcription_api)
    transcription= pipe(output_file, generate_kwargs={"task": "translate"})
    print(transcription["chunks"])    
    transcription_end=time.time()
    print(f"transcription time: {transcription_end-transcription_start} sec")
    data_heading="english"
    return_api(id,data_heading,transcription["chunks"],transcription_api)
    
def return_api(id,transcription_t,transcription,ngrok_url):    
    payload = {
       "id": id,
       transcription_t: transcription
    }

   # Make the POST request
    try:
        response = requests.post(ngrok_url, json=payload)

    # Check if the request was successful
        if response.status_code == 200:
            print("Response from API:")
            print(response.json())  # Print the JSON response
        else:
            print(f"API call failed with status code {response.status_code}")
            print("Response:", response.text)

    except requests.exceptions.RequestException as e:
        print("Error during the API call:", e)


## Section contains the function for the deletion of downloaded audio files

In [9]:
import os
import shutil
def delete():
    working_dir = "/kaggle/working/"
    
    # Iterate over all files and delete them
    for file_name in os.listdir(working_dir):
        file_path = os.path.join(working_dir, file_name)
        
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Delete file or symbolic link
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Delete directory and contents
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")
    
    print("All files in /kaggle/working/ have been deleted.")


# This is the main section that will provide the link to Ngrok

In [ ]:
from flask import Flask, request, jsonify, render_template_string
import threading
from pyngrok import ngrok
from flask_cors import CORS  # Import CORS
import gdown
import os

app = Flask(__name__)
CORS(app)  # Enable CORS for the entire app

# Global variable to store the ngrok URL
public_url = None

def log_error(error_message):
    """Log errors in a controlled manner."""
    print(f"Error: {error_message}")

@app.route('/')
def index():
    """Serve a simple HTML page with the ngrok URL and a copy button."""
    if public_url:
        return render_template_string('''
            <h1>ngrok URL</h1>
            <p id="ngrok-url">{{ public_url }}</p>
            <button onclick="copyToClipboard()">Copy ngrok URL</button>
            <script>
                function copyToClipboard() {
                    const url = document.getElementById('ngrok-url').innerText;
                    navigator.clipboard.writeText(url).then(() => {
                        alert('URL copied to clipboard!');
                    }).catch(err => {
                        alert('Failed to copy URL: ' + err);
                    });
                }
            </script>
        ''', public_url=public_url)
    else:
        return "ngrok URL is not available."

@app.route('/translate', methods=['POST'])
def translate():
    try:
        data = request.json
        print("Received Data:", data)
        result = "Successfully Received"
        id = data.get('videoId')
        url = data.get('videoUrl')
        ngrokurl = data.get('serverUrl')
        delete()
        thread = threading.Thread(target=transcription_api, args=(id, url, ngrokurl))
        thread.start()
        print("Thread started, main program continues.")
        return jsonify({'result': result})
    except Exception as e:
        log_error(str(e))
        return jsonify({'error': 'An error occurred during translation.'}), 500

@app.route('/translate-video', methods=['POST'])
def translate_video():
    try:
        data = request.json
        print("Received Data:", data)
        result = "Successfully Received"
        id = data.get('videoId')
        url = data.get('videoUrl')
        ngrokurl = data.get('serverUrl')
        delete()
        output_video_path = "/kaggle/working/temp_video.mp4"
        output_audio_path = "/kaggle/working/temp_audio.wav"

        # Download the video from Google Drive
        gdown.download(url, output_video_path, quiet=True)

        # Convert video to WAV using ffmpeg
        os.system(f"ffmpeg -i {output_video_path} -ac 1 -ar 16000 {output_audio_path}")

        thread = threading.Thread(target=transcribe, args=(id, output_audio_path, ngrokurl))
        thread.start()
        print("Thread started, main program continues.")
        return jsonify({'result': result})
    except Exception as e:
        log_error(str(e))
        return jsonify({'error': 'An error occurred during video translation.'}), 500

# Set up ngrok
try:
    ngrok.set_auth_token('2of4U3njDabcq5MxYhmR1KF0BWR_653Ac1WWPFPht8xYuPwDN')  # Replace with your ngrok authtoken
    public_url = ngrok.connect(5000).public_url
    # print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

    # Display a button in the Kaggle Notebook to copy the ngrok URL
    display(HTML(f'''
        <p>Copy The ngrok URL: {public_url}</p>
        <textarea id="ngrok-url" style="width: 300px; height: 30px;" readonly>{public_url}</textarea>
        <br><br>
        <button onclick="copyUrl()">Copy ngrok URL</button>
        <script>
            function copyUrl() {{
                const textarea = document.getElementById('ngrok-url');
                textarea.select();
                document.execCommand('copy');
                alert('URL copied to clipboard!');
            }}
        </script>
    '''))
except Exception as e:
    log_error(f"Failed to set up ngrok: {str(e)}")

if __name__ == '__main__':
    try:
        app.run(port=5000)
    except Exception as e:
        log_error(f"Failed to start Flask app: {str(e)}")

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [20]:
# from flask import Flask, request, jsonify
# import threading
# from pyngrok import ngrok
# from flask_cors import CORS  # Import CORS

# app = Flask(__name__)
# CORS(app)  # Enable CORS for the entire app

# @app.route('/translate', methods=['POST'])
# def translate():
#     data = request.json
#     print("Received Data:", data)
#     result = "Successfully Recieved"
#     id = data.get('videoId')
#     url = data.get('videoUrl')
#     ngrokurl = data.get('serverUrl')
#     delete()
#     thread = threading.Thread(target=transcription_api,args=(id,url,ngrokurl))
#     thread.start()
#     print("Thread started, main program continues.")
#     return jsonify({'result': result})

# @app.route('/translate-video', methods=['POST'])
# def translate_video():
#     data = request.json
#     print("Received Data:", data)
#     result = "Successfully Recieved"
#     id = data.get('videoId')
#     url = data.get('videoUrl')
#     ngrokurl = data.get('serverUrl')
#     delete()
#     output_video_path = "/kaggle/working/temp_video.mp4"
#     output_audio_path = "/kaggle/working/temp_audio.wav"

# # Download the video from Google Drive
#     gdown.download(url, output_video_path, quiet=False)

# # Convert video to WAV using ffmpeg
#     os.system(f"ffmpeg -i {output_video_path} -ac 1 -ar 16000 {output_audio_path}")

#     thread = threading.Thread(target=transcribe,args=(id,output_audio_path,ngrokurl))
#     thread.start()
#     print("Thread started, main program continues.")
#     return jsonify({'result': result})

# # Set up ngrok
# ngrok.set_auth_token('2of4U3njDabcq5MxYhmR1KF0BWR_653Ac1WWPFPht8xYuPwDN')  # Replace with your ngrok authtoken
# public_url = ngrok.connect(5000)
# print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

# if __name__ == '__main__':
#     app.run(port=5000)


# Only for testing try not to run this one 

In [21]:
# import gdown
# import os

# output_video_path = "/kaggle/working/temp_video.mp4"
# output_audio_path = "/kaggle/working/temp_audio.wav"
# url="https://res.cloudinary.com/dk06hi9th/video/upload/v1740911234/videos/u5dlgdrgsiq6v8ygx9na.mp4"
# # Download the video from Google Drive
# ngrokurl=""
# id=""
# gdown.download(url, output_video_path, quiet=False)

# # Convert video to WAV using ffmpeg
# os.system(f"ffmpeg -i {output_video_path} -ac 1 -ar 16000 {output_audio_path}")
# thread = threading.Thread(target=transcription,args=(id,output_audio_path,ngrokurl))
# thread.start()
# print("Thread started, main program continues.")
  